In [342]:
import csv
import pandas as pd 
import gspread 
import os
import sys
from gspread_formatting import *
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [454]:
#connect to your service account
scope = ['https://spreadsheets.google.com/feeds'] 
#IMPORTANT!!! replace credentials with your own 
cred = input("What is the name of your json file? ") # 'first-planet-279009-348e07876b8e.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(cred, scope) 
gc = gspread.authorize(credentials)
keyy = input("What is your spreadsheet key (characters after /d/)? ") # '1ERmt36vaEaBbMpKeuMSrKJawOjew141Ppi5jZLRzYjU'
spreadsheet_key = keyy
book = gc.open_by_key(spreadsheet_key)

What is the name of your json file? first-planet-279009-348e07876b8e.json
What is your spreadsheet key (numbers after /d/)? 1ERmt36vaEaBbMpKeuMSrKJawOjew141Ppi5jZLRzYjU


In [455]:
# prep cells

def traffic_light(x):
    s = 0
    t = float(x['Surface Skin Temperature (35-38)'])
    hr = int(x['Heart Rate (>50)'])
    rr = int(x['Respiration Rate (12-16)'])
    
    if 35 <= t <= 38:
        s += 2
    elif 34 <= t <= 39:
        s += 1
    if hr >= 50:
        s += 2
    elif hr >= 40:
        s += 1
    if 12 <= rr <= 16:
        s += 2
    elif 10 <= rr <= 18:
        s += 1
        
    if s >= 5: return "Green"
    elif 3 <= s < 5: return "Orange"
    else: return "Red"

def clean_calls(txt):

    see = []
    
    with open(txt) as x:
        reader = csv.reader(x)
        for row in reader:
            see.append(row)
            
    data = []

    exclusions = ['and', 'but']

    location_indicators = ['in', 'region', 'mrt']
    location_words = ['bishan', 'yishun', 'kranji', 'clementi', 'pasirris', 'changi']

    number_words = {"one" : 1, "two" : 2, "three": 3, "four": 4, "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9, "zero": 0, "ten": 10,
                   'first': 1, 'second': 2, 'third': 3, 'fourth': 4, 'fifth': 5, 'sixth': 6, 'seventh': 7, 'eighth': 8, 'ninth': 9, "tenth": 10}
    numbers = ["1",'2','3','4','5','6','7','8','9','0','10']

    fire_origin_indicators = ['at', 'originating', 'coming', 'on']
    fire_origin_words = ['block', 'carpark', "car", 'park', 'busstop', 'bus stop', 'apartment', 'floor', 'void', 'deck']

    occupants_indicators = ['people', "humans", 'guys', 'girls']
    # occupants_words are number_words and numbers


    for z in see:

        ind = {'Location':'',
              'Fire Origin':'',
              'Number of Occupants': '',
              'Fear Level': ''}

        w = z[0].split(":")[1].split()

        for x in range(len(w)):
            if w[x] in location_indicators:
                if x < 5 and len(w) < x+5: 
                    for y in w:
                        if y.lower() in location_words:
                            ind['Location'] = y.title()

                elif x < 5:
                    for y in w[:x+5]:
                        if y.lower() in location_words:
                            ind['Location'] = y.title()

                elif len(w) < x+5:
                    for y in w[x-5:]:
                        if y.lower() in location_words:
                            ind['Location'] = y.title()

                else:
                    for y in w[x-5:x+5]:
                        if y.lower() in location_words:
                            ind['Location'] = y.title()

            if w[x] in fire_origin_indicators:
                if x < 10 and len(w) < x+10: 
                    for y in w:
                        if y.lower() in fire_origin_words:
                            ind['Fire Origin'] += y.title() + " "
                        if y.lower() in number_words:
                            ind['Fire Origin'] += str(number_words[y.lower()]) + " "
                        if y.lower() in numbers:
                            ind['Fire Origin'] += str(y) + " "

                elif x < 10:
                    for y in w[:x+10]:
                        if y.lower() in fire_origin_words:
                            ind['Fire Origin'] += y.title() + " "
                        if y.lower() in number_words:
                            ind['Fire Origin'] += str(number_words[y.lower()]) + " "
                        if y.lower() in numbers:
                            ind['Fire Origin'] += str(y) + " "

                elif len(w) < x+10:
                    for y in w[x-10:]:
                        if y.lower() in fire_origin_words:
                            ind['Fire Origin'] += y.title() + " "
                        if y.lower() in number_words:
                            ind['Fire Origin'] += str(number_words[y.lower()]) + " "
                        if y.lower() in numbers:
                            ind['Fire Origin'] += str(y) + " "

                else:
                    for y in w[x-10:x+10]:
                        if y.lower() in fire_origin_words:
                            ind['Fire Origin'] += y.title() + " "
                        if y.lower() in number_words:
                            ind['Fire Origin'] += str(number_words[y.lower()]) + " "
                        if y.lower() in numbers:
                            ind['Fire Origin'] += str(y) + " "

            if w[x] in occupants_indicators:
                if x < 3 and len(w) < x+3: 
                    for y in w:
                        if y.lower() in number_words:
                            ind['Number of Occupants'] += str(number_words[y.lower()]) + " "
                        if y.lower() in numbers:
                            ind['Number of Occupants'] += str(y) + " "

                elif x < 3:
                    for y in w[:x+3]:
                        if y.lower() in number_words:
                            ind['Number of Occupants'] += str(number_words[y.lower()]) + " "
                        if y.lower() in numbers:
                            ind['Number of Occupants'] += str(y) + " "

                elif len(w) < x+3:
                    for y in w[x-3:]:
                        if y.lower() in number_words:
                            ind['Number of Occupants'] += str(number_words[y.lower()]) + " "
                        if y.lower() in numbers:
                            ind['Number of Occupants'] += str(y) + " "

                else:
                    for y in w[x-3:x+3]:
                        if y.lower() in number_words:
                            ind['Number of Occupants'] += str(number_words[y.lower()]) + " "
                        if y.lower() in numbers:
                            ind['Number of Occupants'] += str(y) + " "


        score = float(z[1].split(":")[-1])
        if 0.8 < score:
            ind['Fear Level'] = "High"
        elif 0.5 < score:
            ind['Fear Level'] = "Moderate"
        else:
            ind['Fear Level'] = "Low"

        ind['Location'] = ind['Location'].strip()
        ind['Fire Origin'] = ind['Fire Origin'].strip(ind["Number of Occupants"])
        ind['Number of Occupants'] = ind['Number of Occupants'].strip()



        data.append(ind)
        
    return data
        
        
def clean_sen(txt):
    
    see = []
    
    with open(txt) as x:
        reader = csv.reader(x)
        for row in reader:
            see.append(row)
    
    sens = []

    for d in see:
        i = {}
        i['Location'] = d[7].split(":")[1].split("}")[0]
        i['X'] = float(d[0].split(":")[-1])
        i['Y'] = float(d[1].split(":")[-1])
        i['Tag'] = d[2].split(":")[-1].strip('"')
        i['Temp'] = float(d[3].split(":")[-1])
        smoke = float(d[4].split(":")[-1])
        if 1 < smoke:
            i['Smoke Level'] = "Dangerous"
        elif 0.8 < smoke:
            i['Smoke Level'] = "Unhealthy"
        else:
            i['Smoke Level'] = "Moderate"
        i['Humidity'] = float(d[5].split(":")[-1])
        date = d[6].split("T")[0].split(":")[-1].strip('"')
        time = d[6].split("T")[1].split("+")[0].split(".")[0]
        i['Timestamp'] = date + " " + time
        sens.append(i)
    
    return sens
    
for_ff = cellFormat(
    backgroundColor=color(1, 0.604, 0.635),
    textFormat=textFormat(bold=True, foregroundColor=color(0, 0, 0)),
    horizontalAlignment='CENTER'
    )

for_com = cellFormat(
    backgroundColor=color(0.71, 0.918, 0.843),
    textFormat=textFormat(bold=True, foregroundColor=color(0, 0, 0)),
    horizontalAlignment='CENTER'
    )

for_sen = cellFormat(
    backgroundColor=color(0.878, 0.99, 0.99),
    textFormat=textFormat(bold=True, foregroundColor=color(0, 0, 0)),
    horizontalAlignment='CENTER'
    )

act = []
inact = []


In [465]:
def initiate_page():
    kick = book.worksheet('Sheet1')
    book.add_worksheet("Tracker", 100, 100)
    book.del_worksheet(kick)
    worksheet = book.worksheet('Tracker')

def refresh_page():
    hsm_data = []
    com_data = []
    sen_data = []

    cases = worksheet.get_all_records()
    for x in cases:
        if x['Active'] not in act and x['Active']:
            act.append(x["Active"])
        if x['Inactive'] not in inact and x['Inactive']:
            inact.append(x["Inactive"])

    for x in inact:
        if x in act:
            act.remove(x)

    directory = os.getcwd()

    for r,d,f in os.walk(directory):
        for file in f:
            if file.endswith('.csv'):
                if file.startswith('hsm'):
                    with open(file, encoding = 'utf-8-sig') as x:
                        reader = csv.DictReader(x)
                        for row in reader:
                            hsm_data.append(dict(row))
                if file.startswith('com_'):
                    with open(file, encoding = 'utf-8-sig') as x:
                        reader = csv.DictReader(x)
                        for row in reader:
                            com_data.append(dict(row))
            if file.endswith('.txt'):
                if file.startswith('com_'):
                    com_data += clean_calls(file)
                if file.startswith('sen_'):
                    sen_data += clean_sen(file)


    for c in hsm_data:
        if c['Location'] not in act and c['Location'] not in inact:
            act.append(c['Location'])
            com = c['Location'] + "_Com"
            ff = c['Location'] + "_FF"
            sen = c['Location'] + "_Sen"
            book.add_worksheet(sen, 100, 100)
            book.add_worksheet(ff, 100, 100)
            book.add_worksheet(com, 100, 100)                  

    for c in com_data:
        if c['Location'] not in act and c['Location'] not in inact:
            act.append(c['Location'])
            sen = c['Location'] + "_Sen"
            ff = c['Location'] + "_FF"
            com = c['Location'] + "_Com"
            book.add_worksheet(sen, 100, 100)
            book.add_worksheet(ff, 100, 100)
            book.add_worksheet(com, 100, 100)


    for c in sen_data:
        if c['Location'] not in act and c['Location'] not in inact:
            act.append(c['Location'])
            com = c['Location'] + "_Com"
            ff = c['Location'] + "_FF"
            sen = c['Location'] + "_Sen"
            book.add_worksheet(sen, 100, 100)
            book.add_worksheet(ff, 100, 100)
            book.add_worksheet(com, 100, 100)

    for a in act: 
        sen = a + "_Sen"
        ff = a + "_FF"
        com = a + "_Com"

        cur = book.worksheet(ff)
        cur.clear()
        new = []
        for c in hsm_data:
            if c['Location'] == a:
                new.append(c)

        if new:
            for x in new:
                x['Traffic Light Status'] = traffic_light(x)

            real = []
            for x in new:
                if x['Traffic Light Status'] == "Red":
                    real.append(x)
            for x in new:
                if x['Traffic Light Status'] == "Orange":
                    real.append(x)
            for x in new:
                if x['Traffic Light Status'] == "Green":
                    real.append(x)        

            addin = pd.DataFrame(real)
            d2g.upload(addin, spreadsheet_key, ff, credentials=credentials, row_names=False)
            format_cell_range(cur, 'A1:F1', for_ff)
            set_column_width(cur, 'C:F', 220)

        cur = book.worksheet(com)
        cur.clear()
        new = []
        for c in com_data:
            if c['Location'] == a:
                new.append(c)
        if new:    
            addin = pd.DataFrame(new)
            d2g.upload(addin, spreadsheet_key, com, credentials=credentials, row_names=False)
            format_cell_range(cur, "A1:D1", for_com)
            set_column_width(cur, 'B', 300)
            set_column_width(cur, 'C', 300)

        cur = book.worksheet(sen)
        cur.clear()
        new = []
        for c in sen_data:
            if c['Location'] == a:
                new.append(c)
        if new:     
            addin = pd.DataFrame(new)
            d2g.upload(addin, spreadsheet_key, sen, credentials=credentials, row_names=False)
            format_cell_range(cur, 'A1:H1', for_sen)
            set_column_width(cur, 'H', 200)

    active_ones = pd.DataFrame(act, columns = ['Active'])
    inactive_ones = pd.DataFrame(inact,  columns = ['Inactive'])

    cases_list = pd.concat([active_ones,inactive_ones], axis=1)
    cases_list = cases_list.fillna('')

    d2g.upload(cases_list, spreadsheet_key, "Tracker", credentials=credentials, row_names=False)
    format_cell_range(worksheet, "1", bold)
    format_cell_range(worksheet, "A1", for_com)
    format_cell_range(worksheet, "B1", for_ff)


    for x in inact:
        sen = x + "_Sen"
        ff = x + "_FF"
        com = x + "_Com"
        try:
            kick1 = book.worksheet(sen)
            kick2 = book.worksheet(ff)
            kick3 = book.worksheet(com)
            book.del_worksheet(kick1)
            book.del_worksheet(kick2)
            book.del_worksheet(kick3)
        except:
            continue

In [464]:
# cell to start/refresh the worksheet
refresh_page()

In [463]:
# cell to restart the worksheet
inact = ['Bishan', 'Kranji', 'Yishun']
refresh_page()
worksheet.clear()
inact = []
act = []

In [ ]:
#if code doesn't work, run this and try again
#!pip install gspread-formatting
#!pip install oauth2client
#!pip install df2gspread